In [6]:
%reload_ext autoreload
%autoreload 2
%autosave 0

import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

import os
import sys
import numpy as np
import pandas as pd
import gmsh_api
import gmsh_api.gmsh as gmsh
print("gmsh_api: v{}".fromat(gmsh_api.__version__))
print("GMSH_API_VERSION: v{}".format(gmsh.GMSH_API_VERSION))

Autosave disabled
0.1.1
GMSH_API_VERSION:4.2


In [2]:
gmsh.initialize()

gmsh.option.setNumber("General.Terminal", 1)
# gmsh.option.setNumber("Mesh.Algorithm", 5) # delquad
# gmsh.option.setNumber("Mesh.RecombineAll", 1)

gmsh.model.add("square")
gmsh.model.geo.addPoint(0, 0, 0, 0.6, 1)
gmsh.model.geo.addPoint(1, 0, 0, 0.6, 2)
gmsh.model.geo.addPoint(1, 1, 0, 0.1, 3)
gmsh.model.geo.addPoint(0, 1, 0, 0.1, 4)
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
# try automatic assignement of tag
line4 = gmsh.model.geo.addLine(4, 1)
gmsh.model.geo.addCurveLoop([1, 2, 3, line4], 1)
gmsh.model.geo.addPlaneSurface([1], 6)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("square.unv")


In [3]:
mesh = gmsh_api.Mesh.from_gmsh(gmsh)
mesh

{1: 0, 2: 1, 15: 2}


/home/ingolf.lepenies/anaconda3/envs/sfem/lib/python2.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


<gmsh_api.Mesh instance at 0x7f0d4f729b90>

'4.2'

In [5]:
gmsh.__file__

'/home/ingolf.lepenies/projects/gmsh_api/gmsh_api/gmsh.pyc'